# Predicting March Madness winners using automated feature engineering and head to head information

In [4]:
#this is my standard way of handling path issues and shared libraries with Jupyter Notebooks
#Basically just set the path using the folder_name of the project and create any env variables
import sys,os
folder_name="march_madness"
BASE_DIR=os.path.abspath(".").split(folder_name)[0]+folder_name
DATA_ROOT=os.path.join(BASE_DIR,"data")
sys.path.insert(0, DATA_ROOT)

import utils
#use reload to easily reload changes
from importlib import reload
_=reload(utils)
import pandas as pd
np=pd.np
pd.options.display.max_rows=10#So we don't have to always write head
pd.options.display.max_columns=999
from sklearn.ensemble import *
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import featuretools as ft
ft.__version__

'0.6.1'

# Load Data

First build team game logs as the base for building the matchups and team_game entiysets

In [5]:
reload(utils)
team_game_log=utils.build_team_game_log()
matchups_es=utils.load_matchups_entityset(team_game_log)
matchups_es
matchup_logs=matchups_es['matchup_log'].df


In [6]:
import datetime

In [7]:
tourney_starts=matchup_logs[matchup_logs['game_type']=='march_madness'].groupby("season").first()['game_datetime']
tourney_starts=tourney_starts.append(pd.Series(datetime.datetime(2019,3,21),index=[2019]))
tourney_starts

2003   2003-03-18
2004   2004-03-16
2005   2005-03-15
2006   2006-03-14
2007   2007-03-13
          ...    
2015   2015-03-17
2016   2016-03-15
2017   2017-03-14
2018   2018-03-13
2019   2019-03-21
Length: 17, dtype: datetime64[ns]

In [8]:
from featuretools.primitives.base  import (
    AggregationPrimitive)
from featuretools.variable_types.variable import Variable
class LastFixed(AggregationPrimitive):
    """Returns the last value."""
    name = "last"
    input_types = [Variable]
    return_type = None
    stack_on_self = False

    def get_function(self):
        def pd_last(x):
            if len(x)>1:
                if pd.isnull(x.iloc[-1]):
                    try:
                        return x.dropna().iloc[-1]
                    except:
                        return pd.np.nan
                else:
                    return x.iloc[-1]
            return x.iloc[-1]
        return pd_last
AGG_DEFAULTS=["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
agg_primitives=AGG_DEFAULTS+[LastFixed,"median"]

In [9]:
#clf=baseline_clf

In [10]:
tourney_labels=utils.make_tourney_labels(matchups_es, tourney_starts, only_matchups_with_history=False)
tourney_labels['ID']=tourney_labels['game_index'].apply(utils.game_index_to_kaggle_id)
tourney_labels['Pred']=pd.np.nan#clf.predict_proba(X[baseline_features])[:,1]
tourney_labels

,game_index,time,team_1_wins,ID,Pred
0,2003_134_1411_1421,2003-03-18,False,2003_1411_1421,NaN
1,2003_136_1112_1436,2003-03-20,True,2003_1112_1436,NaN
2,2003_136_1113_1272,2003-03-20,True,2003_1113_1272,NaN
3,2003_136_1140_1163,2003-03-20,False,2003_1140_1163,NaN
4,2003_136_1141_1166,2003-03-20,True,2003_1141_1166,NaN
...,...,...,...,...,...
1043,2018_146_1181_1242,2018-03-25,False,2018_1181_1242,NaN
1044,2018_146_1403_1437,2018-03-25,False,2018_1403_1437,NaN
1045,2018_152_1242_1437,2018-03-31,False,2018_1242_1437,NaN
1046,2018_152_1260_1276,2018-03-31,False,2018_1260_1276,NaN


In [11]:
possible_tourney_games_df=utils.build_possible_tourney_games()
predictions=possible_tourney_games_df.merge(tourney_labels,on=["ID"], how="left")
predictions['Pred']=predictions['Pred'].fillna(.5)
predictions

,Team1_ID,Team2_ID,Season,TeamID,ID,game_index,time,team_1_wins,Pred
0,1107,1110,2014,1107,2014_1107_1110,NaN,NaT,NaN,0.5
1,1107,1112,2014,1107,2014_1107_1112,NaN,NaT,NaN,0.5
2,1107,1113,2014,1107,2014_1107_1113,NaN,NaT,NaN,0.5
3,1107,1124,2014,1107,2014_1107_1124,NaN,NaT,NaN,0.5
4,1107,1140,2014,1107,2014_1107_1140,NaN,NaT,NaN,0.5
...,...,...,...,...,...,...,...,...,...
13663,1449,1459,2019,1449,2019_1449_1459,NaN,NaT,NaN,0.5
13664,1449,1463,2019,1449,2019_1449_1463,NaN,NaT,NaN,0.5
13665,1458,1459,2019,1458,2019_1458_1459,NaN,NaT,NaN,0.5
13666,1458,1463,2019,1458,2019_1458_1463,NaN,NaT,NaN,0.5


In [12]:
def get_tourney_start(row):
    return tourney_starts[row['Season']]#+pd.Timedelta(days=1)
predictions['time']=predictions.apply(get_tourney_start,axis=1)
predictions['team_game_index']=predictions['ID']+"_"+predictions['Team1_ID'].astype(str)
predictions=predictions.rename(columns={"team_1_wins":"team_wins"})
prediction_labels=predictions[['team_game_index','time','team_wins']]
prediction_labels=prediction_labels.rename(columns={"time":"game_datetime"})
prediction_labels['season']=prediction_labels['team_game_index'].apply(lambda x: int(x.split("_")[0]))
prediction_labels['team_TeamID']=prediction_labels['team_game_index'].apply(lambda x: int(x.split("_")[1]))
prediction_labels['opp_TeamID']=prediction_labels['team_game_index'].apply(lambda x: int(x.split("_")[2]))
prediction_labels['team_season_id']=prediction_labels['season'].astype(str)+"_"+prediction_labels['team_TeamID'].astype(str)
prediction_labels['opp_season_id']=prediction_labels['season'].astype(str)+"_"+prediction_labels['opp_TeamID'].astype(str)
prediction_labels['team_game_loc']='N'
tourney_seeds = utils.build_tourney_seeds()
merge_cols = ['season_id', 'region', 'seed']
tourney_seeds = tourney_seeds.rename(columns={"team_season_id": "season_id"})
prediction_labels=prediction_labels.merge(
    tourney_seeds[merge_cols].add_prefix("team_"),on="team_season_id", how="left").merge(
    tourney_seeds[merge_cols].add_prefix("opp_"), on="opp_season_id", how="left")
prediction_labels['team_is_higher_seed']=prediction_labels['team_seed']>prediction_labels['opp_seed']
prediction_labels=prediction_labels[prediction_labels['season']==2019]
prediction_labels

,team_game_index,game_datetime,team_wins,season,team_TeamID,opp_TeamID,team_season_id,opp_season_id,team_game_loc,team_region,team_seed,opp_region,opp_seed,team_is_higher_seed
11390,2019_1101_1113_1101,2019-03-21,NaN,2019,1101,1113,2019_1101,2019_1113,N,Y,15,X,12,True
11391,2019_1101_1120_1101,2019-03-21,NaN,2019,1101,1120,2019_1101,2019_1120,N,Y,15,Y,5,True
11392,2019_1101_1124_1101,2019-03-21,NaN,2019,1101,1124,2019_1101,2019_1124,N,Y,15,X,9,True
11393,2019_1101_1125_1101,2019-03-21,NaN,2019,1101,1125,2019_1101,2019_1125,N,Y,15,W,12,True
11394,2019_1101_1133_1101,2019-03-21,NaN,2019,1101,1133,2019_1101,2019_1133,N,Y,15,W,15,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13663,2019_1449_1459_1449,2019-03-21,NaN,2019,1449,1459,2019_1449,2019_1459,N,Y,9,Y,7,True
13664,2019_1449_1463_1449,2019-03-21,NaN,2019,1449,1463,2019_1449,2019_1463,N,Y,9,W,14,False
13665,2019_1458_1459_1458,2019-03-21,NaN,2019,1458,1459,2019_1458,2019_1459,N,Z,5,Y,7,False
13666,2019_1458_1463_1458,2019-03-21,NaN,2019,1458,1463,2019_1458,2019_1463,N,Z,5,W,14,False


In [13]:
prediction_game_logs=team_game_log.append(prediction_labels,sort=False).copy().sort_values(by=["game_datetime","team_game_index"])
prediction_game_logs

,season,DayNum,team_TeamID,team_Score,opp_TeamID,opp_Score,NumOT,team_FGM,team_FGA,team_FGM3,team_FGA3,team_FTM,team_FTA,team_OR,team_DR,team_Ast,team_TO,team_Stl,team_Blk,team_PF,opp_FGM,opp_FGA,opp_FGM3,opp_FGA3,opp_FTM,opp_FTA,opp_OR,opp_DR,opp_Ast,opp_TO,opp_Stl,opp_Blk,opp_PF,game_type,game_index,game_datetime,team_margin,opp_margin,team_wins,team_wins_int,opp_wins,opp_wins_int,team_game_loc,matchup_id,team_season_id,opp_season_id,team_game_index,opp_game_index,game_end,game_PF,game_MOV,opp_poss,opp_FGA2,opp_FGM2,opp_PFP,opp_TSA,opp_TS,opp_FG2_points,opp_FG3_points,opp_FG_points,opp_FG2_points_share_FG,opp_FG2_points_share_points,opp_FG3_points_share_FG,opp_FG3_points_share_points,opp_FG_points_share_FG,opp_FG_points_share_points,opp_FG_points/FT_points,opp_FT_point_share,opp_eFGP,opp_scoring_eff,opp_score_op,opp_off_rtg,opp_IE_numerator,opp_FGP3,opp_FTP,opp_FGP,opp_FGP2,team_poss,team_FGA2,team_FGM2,team_PFP,team_TSA,team_TS,team_FG2_points,team_FG3_points,team_FG_points,team_FG2_points_share_FG,team_FG2_points_share_points,team_FG3_points_share_FG,team_FG3_points_share_points,team_FG_points_share_FG,team_FG_points_share_points,team_FG_points/FT_points,team_FT_point_share,team_eFGP,team_scoring_eff,team_score_op,team_off_rtg,team_IE_numerator,team_FGP3,team_FTP,team_FGP,team_FGP2,opp_def_rtg,opp_sos,opp_to_poss,opp_orb_pct,opp_drb_pct,opp_reb_pct,opp_IE,opp_ast_rtio,opp_Blk_pct,opp_Stl_pct,team_def_rtg,team_sos,team_to_poss,team_orb_pct,team_drb_pct,team_reb_pct,team_IE,team_ast_rtio,team_Blk_pct,team_Stl_pct,team_region,team_seed,opp_region,opp_seed,team_is_higher_seed
0,2003,10.0,1104,68.0,1328,62.0,0.0,27.0,58.0,3.0,14.0,11.0,18.0,14.0,24.0,13.0,23.0,7.0,1.0,22.0,22.0,53.0,2.0,10.0,16.0,22.0,10.0,22.0,8.0,18.0,9.0,2.0,20.0,regular_season,2003_10_1104_1328,2002-11-14,6.0,-6.0,True,1.0,False,0.0,N,1104_1328,2003_1104,2003_1328,2003_10_1104_1328_1104,2003_10_1104_1328_1328,2002-11-15,42.0,6.0,71.450,43.0,20.0,0.476190,62.68,0.494576,40.0,6.0,46.0,0.869565,0.645161,0.130435,0.096774,1.0,0.741935,2.875000,0.258065,0.433962,0.977147,0.888034,86.773968,32.0,0.200000,0.727273,0.415094,0.465116,75.550,44.0,24.0,0.523810,65.92,0.515777,48.0,9.0,57.0,0.842105,0.705882,0.157895,0.132353,1.0,0.838235,5.181818,0.161765,0.491379,1.021788,0.880874,90.006618,36.5,0.214286,0.611111,0.465517,0.545455,90.006618,-3.232650,0.251924,0.294118,0.611111,0.452614,0.467153,8.943544,4.545455,11.912641,86.773968,3.232650,0.304434,0.388889,0.705882,0.547386,0.532847,12.676743,2.325581,9.797061,Y,10.0,W,1.0,True
1,2003,10.0,1328,62.0,1104,68.0,0.0,22.0,53.0,2.0,10.0,16.0,22.0,10.0,22.0,8.0,18.0,9.0,2.0,20.0,27.0,58.0,3.0,14.0,11.0,18.0,14.0,24.0,13.0,23.0,7.0,1.0,22.0,regular_season,2003_10_1104_1328,2002-11-14,-6.0,6.0,False,0.0,True,1.0,N,1104_1328,2003_1328,2003_1104,2003_10_1104_1328_1328,2003_10_1104_1328_1104,2002-11-15,42.0,6.0,75.550,44.0,24.0,0.523810,65.92,0.515777,48.0,9.0,57.0,0.842105,0.705882,0.157895,0.132353,1.0,0.838235,5.181818,0.161765,0.491379,1.021788,0.880874,90.006618,36.5,0.214286,0.611111,0.465517,0.545455,71.450,43.0,20.0,0.476190,62.68,0.494576,40.0,6.0,46.0,0.869565,0.645161,0.130435,0.096774,1.0,0.741935,2.875000,0.258065,0.433962,0.977147,0.888034,86.773968,32.0,0.200000,0.727273,0.415094,0.465116,86.773968,3.232650,0.304434,0.388889,0.705882,0.547386,0.532847,12.676743,2.325581,9.797061,90.006618,-3.232650,0.251924,0.294118,0.611111,0.452614,0.467153,8.943544,4.545455,11.912641,W,1.0,Y,10.0,False
2,2003,10.0,1272,70.0,1393,63.0,0.0,26.0,62.0,8.0,20.0,10.0,19.0,15.0,28.0,16.0,13.0,4.0,4.0,18.0,24.0,67.0,6.0,24.0,9.0,20.0,20.0,25.0,7.0,12.0,8.0,6.0,16.0,regular_season,2003_10_1272_1393,2002-11-14,7.0,-7.0,True,1.0,False,0.0,N,1272_1393,2003_1272,2003_1393,2003_10_1272_1393_1272,2003_10_1272_1393_1393,2002-11-15,34.0,7.0,68.500,43.0,18.0,0.470588,75.80,0.415567,36.0,18.0,54.0,0.666667,0.571429,0.333333,0.285714,1.0,0.857143,6.000000,0.142857,0.402985,0.823529,1.116788,91.970803,34.0,0.250000,0.450000,0.358209,0.418605,69.

In [14]:
preds_entityset=utils.load_team_game_logs_entityset(prediction_game_logs)

In [15]:
training_window=ft.Timedelta("52 weeks")#more sophisticated approaches could be explored
feature_matrix, features = ft.dfs(target_entity="team_game_log", 
                                  cutoff_time=prediction_labels[prediction_labels['season']==2019],
                                  training_window=training_window,
                                  entityset=preds_entityset,
                                  cutoff_time_in_index=True,
                                  verbose=True,
                                  drop_contains=["matchup_id",
                                              "team_game_index","team_season_id",
                                             "opp_game_index","opp_season_id",
                                              "team_TeamID","opp_TeamID","game_index",'TeamName'
                                             ],
                                      trans_primitives=[],
                                      agg_primitives=agg_primitives,
                                 )
# encode categorical values
team_game_log_fm_encoded, features_encoded = ft.encode_features(feature_matrix,
                                                  features)
team_game_log_fm_encoded

Built 1019 features
Elapsed: 00:18 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


DayNum  team_Score  opp_Score  team_FGM  \
team_game_index     time                                                  
2019_1101_1113_1101 2019-03-21     NaN         NaN        NaN       NaN   
2019_1101_1120_1101 2019-03-21     NaN         NaN        NaN       NaN   
2019_1101_1124_1101 2019-03-21     NaN         NaN        NaN       NaN   
2019_1101_1125_1101 2019-03-21     NaN         NaN        NaN       NaN   
2019_1101_1133_1101 2019-03-21     NaN         NaN        NaN       NaN   
...                                ...         ...        ...       ...   
2019_1449_1459_1449 2019-03-21     NaN         NaN        NaN       NaN   
2019_1449_1463_1449 2019-03-21     NaN         NaN        NaN       NaN   
2019_1458_1459_1458 2019-03-21     NaN         NaN        NaN       NaN   
2019_1458_1463_1458 2019-03-21     NaN         NaN        NaN       NaN   
2019_1459_1463_1459 2019-03-21     NaN         NaN        NaN       NaN   

                                team_FGA  team_FGM3  team_FGA3  team_FTM  \
team_game_index     time                                                   
2019_1101_1113_1101 2019-03-21       NaN        NaN        NaN       NaN   
2019_1101_1120_1101 2019-03-21       NaN        NaN        NaN       NaN   
2019_1101_1124_1101 2019-03-21       NaN        NaN        NaN       NaN   
2019_1101_1125_1101 2019-03-21       NaN        NaN        NaN       NaN   
2019_1101_1133_1101 2019-03-21       NaN        NaN        NaN       NaN   
...                                  ...        ...        ...       ...   
2019_1449_1459_1449 2019-03-21       NaN        NaN        NaN       NaN   
2019_1449_1463_1449 2019-03-21       NaN        NaN        NaN       NaN   
2019_1458_1459_1458 2019-03-21       NaN        NaN        NaN       NaN   
2019_1458_1463_1458 2019-03-21       NaN        NaN        NaN       NaN   
2019_1459_1463_1459 2019-03-21       NaN        NaN        NaN       NaN   

                                team_FTA  team_OR  team_DR  team_Ast  team_TO  \
team_game_index     time                                                        
2019_1101_1113_1101 2019-03-21       NaN      NaN      NaN       NaN      NaN   
2019_1101_1120_1101 2019-03-21       NaN      NaN      NaN       NaN      NaN   
2019_1101_1124_1101 2019-03-21       NaN      NaN      NaN       NaN      NaN   
2019_1101_1125_1101 2019-03-21       NaN      NaN      NaN       NaN      NaN   
2019_1101_1133_1101 2019-03-21       NaN      NaN      NaN       NaN      NaN   
...                                  ...      ...      ...       ...      ...   
2019_1449_1459_1449 2019-03-21       NaN      NaN      NaN       NaN      NaN   
2019_1449_1463_1449 2019-03-21       NaN      NaN      NaN       NaN      NaN   
2019_1458_1459_1458 2019-03-21       NaN      NaN      NaN       NaN      NaN   
2019_1458_1463_1458 2019-03-21       NaN      NaN      NaN       NaN      NaN   
2019_1459_1463_1459 2019-03-21       NaN      NaN      NaN       NaN      NaN   

                                team_Stl  team_Blk  team_PF  opp_FGM  opp_FGA  \
team_game_index     time                                                        
2019_1101_1113_1101 2019-03-21       NaN       NaN      NaN      NaN      NaN   
2019_1101_1120_1101 2019-03-21       NaN       NaN      NaN      NaN      NaN   
2019_1101_1124_1101 2019-03-21       NaN       NaN      NaN      NaN      NaN   
2019_1101_1125_1101 2019-03-21       NaN       NaN      NaN      NaN      NaN   
2019_1101_1133_1101 2019-03-21       NaN       NaN      NaN      NaN      NaN   
...                                  ...       ...      ...      ...      ...   
2019_1449_1459_1449 2019-03-21       NaN       NaN      NaN      NaN      NaN   
2019_1449_1463_1449 2019-03-21       NaN       NaN      NaN      NaN      NaN   
2019_1458_1459_1458 2019-03-21       NaN       NaN      NaN      NaN      NaN   
2019_1458_1463_1458 2019-03-21       NaN       NaN      NaN      NaN      NaN   
2019_1459_1463_1459 2019-03-21       NaN 

In [16]:
matchups=pd.DataFrame()
matchups['game_index']=team_game_log_fm_encoded.reset_index()['team_game_index'].apply(
        lambda x: "_".join(x.split("_")[:-1]))
matchups=matchups[['game_index']].drop_duplicates("game_index",keep="first")
matchups.index=team_game_log_fm_encoded.index
matchups

,,game_index
team_game_index,time,
2019_1101_1113_1101,2019-03-21,2019_1101_1113
2019_1101_1120_1101,2019-03-21,2019_1101_1120
2019_1101_1124_1101,2019-03-21,2019_1101_1124
2019_1101_1125_1101,2019-03-21,2019_1101_1125
2019_1101_1133_1101,2019-03-21,2019_1101_1133
...,...,...
2019_1449_1459_1449,2019-03-21,2019_1449_1459
2019_1449_1463_1449,2019-03-21,2019_1449_1463
2019_1458_1459_1458,2019-03-21,2019_1458_1459


In [17]:

reload(utils)
fm_encoded_preds=utils.merge_matchups_and_team_game_log_features(matchups,team_game_log_fm_encoded)
#fm_encoded=fm_encoded.drop_duplicates(subset=["game_index"],keep="first")
fm_encoded_preds

team_game_index       time      game_index  DayNum_tgm_team  \
0     2019_1101_1113_1101 2019-03-21  2019_1101_1113              NaN   
1     2019_1101_1120_1101 2019-03-21  2019_1101_1120              NaN   
2     2019_1101_1124_1101 2019-03-21  2019_1101_1124              NaN   
3     2019_1101_1125_1101 2019-03-21  2019_1101_1125              NaN   
4     2019_1101_1133_1101 2019-03-21  2019_1101_1133              NaN   
...                   ...        ...             ...              ...   
2273  2019_1449_1459_1449 2019-03-21  2019_1449_1459              NaN   
2274  2019_1449_1463_1449 2019-03-21  2019_1449_1463              NaN   
2275  2019_1458_1459_1458 2019-03-21  2019_1458_1459              NaN   
2276  2019_1458_1463_1458 2019-03-21  2019_1458_1463              NaN   
2277  2019_1459_1463_1459 2019-03-21  2019_1459_1463              NaN   

      NumOT is unknown_tgm_team  game_MOV_tgm_team  game_PF_tgm_team  \
0                             1                NaN               NaN   
1                             1                NaN               NaN   
2                             1                NaN               NaN   
3                             1                NaN               NaN   
4                             1                NaN               NaN   
...                         ...                ...               ...   
2273                          1                NaN               NaN   
2274                          1                NaN               NaN   
2275                          1                NaN               NaN   
2276                          1                NaN               NaN   
2277                          1                NaN               NaN   

      game_type is unknown_tgm_team  opp_Ast_tgm_team  opp_Blk_tgm_team  \
0                                 1               NaN               NaN   
1                                 1               NaN               NaN   
2                                 1               NaN               NaN   
3                                 1               NaN               NaN   
4                                 1               NaN               NaN   
...                             ...               ...               ...   
2273                              1               NaN               NaN   
2274                              1               NaN               NaN   
2275                              1               NaN               NaN   
2276                              1               NaN               NaN   
2277                              1               NaN               NaN   

      opp_Blk_pct_tgm_team  opp_DR_tgm_team  opp_FG2_points_tgm_team  \
0                      NaN              NaN                      NaN   
1                      NaN              NaN                      NaN   
2                      NaN              NaN                      NaN   
3                      NaN              NaN                      NaN   
4                      NaN              NaN                      NaN   
...                    ...              ...                      ...   
2273                   NaN              NaN                      NaN   
2274                   NaN              NaN                      NaN   
2275                   NaN              NaN                      NaN   
2276                   NaN              NaN                      NaN   
2277                   NaN              NaN                      NaN   

      opp_FG2_points_share_FG_tgm_team  opp_FG2_points_share_points_tgm_team  \
0                                  NaN                                   NaN   
1                                  NaN                                   NaN   
2                                  NaN                                   NaN   
3                                  NaN                                   NaN   
4                                  NaN                                   NaN   
...                   

In [18]:
reload(utils)
X,y=utils.transform_to_Xy(fm_encoded_preds, prediction_labels.rename(columns={"game_datetime":"time"}))
X

game_index  DayNum_tgm_team  NumOT is unknown_tgm_team  \
0     2019_1101_1113              0.0                          1   
1     2019_1101_1120              0.0                          1   
2     2019_1101_1124              0.0                          1   
3     2019_1101_1125              0.0                          1   
4     2019_1101_1133              0.0                          1   
...              ...              ...                        ...   
2273  2019_1449_1459              0.0                          1   
2274  2019_1449_1463              0.0                          1   
2275  2019_1458_1459              0.0                          1   
2276  2019_1458_1463              0.0                          1   
2277  2019_1459_1463              0.0                          1   

      game_MOV_tgm_team  game_PF_tgm_team  game_type is unknown_tgm_team  \
0                   0.0               0.0                              1   
1                   0.0               0.0                              1   
2                   0.0               0.0                              1   
3                   0.0               0.0                              1   
4                   0.0               0.0                              1   
...                 ...               ...                            ...   
2273                0.0               0.0                              1   
2274                0.0               0.0                              1   
2275                0.0               0.0                              1   
2276                0.0               0.0                              1   
2277                0.0               0.0                              1   

      opp_Ast_tgm_team  opp_Blk_tgm_team  opp_Blk_pct_tgm_team  \
0                  0.0               0.0                   0.0   
1                  0.0               0.0                   0.0   
2                  0.0               0.0                   0.0   
3                  0.0               0.0                   0.0   
4                  0.0               0.0                   0.0   
...                ...               ...                   ...   
2273               0.0               0.0                   0.0   
2274               0.0               0.0                   0.0   
2275               0.0               0.0                   0.0   
2276               0.0               0.0                   0.0   
2277               0.0               0.0                   0.0   

      opp_DR_tgm_team  opp_FG2_points_tgm_team  \
0                 0.0                      0.0   
1                 0.0                      0.0   
2                 0.0                      0.0   
3                 0.0                      0.0   
4                 0.0                      0.0   
...               ...                      ...   
2273              0.0                      0.0   
2274              0.0                      0.0   
2275              0.0                      0.0   
2276              0.0                      0.0   
2277              0.0                      0.0   

      opp_FG2_points_share_FG_tgm_team  opp_FG2_points_share_points_tgm_team  \
0                                  0.0                                   0.0   
1                                  0.0                                   0.0   
2                                  0.0                                   0.0   
3                                  0.0                                   0.0   
4                                  0.0                                   0.0   
...                                ...                                   ...   
2273                               0.0                                   0.0   
2274                               0.0                                   0.0   
2275                               0.0                                   0.0   
2276                               0.0                                   0.0   
2277  

In [37]:
from scipy.stats import logistic
def rating_mapping(rank):
    N=353
    percent_diff=(N-rank)/(N)
    percent_diff=np.clip(percent_diff,.05,.99)
    print(percent_diff)
    return logistic.ppf(percent_diff,scale=50)

def tourney_rating_mapping(rank):
    N=64
    percent_diff=(N-rank)/(N)
    percent_diff=np.clip(percent_diff,.10,.90)
    return logistic.ppf(percent_diff,scale=100)

def predict_prob(rank1,rank2):
    E_1 = 1./(1 + 10 ** ((rank2 - rank1) / (320.)))
    return E_1

In [39]:
massey = pd.read_csv(DATA_ROOT + "/MasseyOrdinals_thru_2019_day_128/MasseyOrdinals_thru_2019_day_128.csv",engine="python")
use_systems=["SAG","POM","NET","DOK","SPR","AP","WOB"]
systems=massey[(massey['SystemName'].isin(use_systems))]

In [43]:

X['season']=X['game_index'].apply(lambda x: int(x.split("_")[0]))
X['team_TeamID']=X['team_TeamID'].astype(int)
X['opp_TeamID']=X['opp_TeamID'].astype(int)
tourney_systems=systems[systems['TeamID'].isin(X['team_TeamID'].unique().tolist()+X['opp_TeamID'].unique().tolist())]
tourney_systems=tourney_systems[tourney_systems['Season']==2019]
tourney_systems

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
3492320,2019,9,AP,1120,9
3492321,2019,9,AP,1138,25
3492323,2019,9,AP,1181,1
3492324,2019,9,AP,1199,14
3492325,2019,9,AP,1211,3
...,...,...,...,...,...
3797171,2019,128,WOB,1439,18
3797179,2019,128,WOB,1449,29
3797188,2019,128,WOB,1458,17
3797189,2019,128,WOB,1459,20


In [44]:
errors=[]
def get_ranking(row):
    d={}
    for name in use_systems:
        for prefix in ["team_","opp_"]:
            team_ranks=tourney_systems[(tourney_systems['Season']==row['season'])&(tourney_systems['TeamID']==row[prefix+'TeamID'])]
            try:
                d[prefix+name]=team_ranks[team_ranks['SystemName']==name].iloc[-1]['OrdinalRank']
            except:
                errors.append(name)
    return pd.Series(d)
rankings_df=X.apply(get_ranking, axis=1)
rankings_df

,opp_AP,opp_DOK,opp_NET,opp_POM,opp_SAG,opp_SPR,opp_WOB,team_AP,team_DOK,team_NET,team_POM,team_SAG,team_SPR,team_WOB
0,17.0,50.0,67.0,62.0,53.0,55.0,46.0,NaN,155.0,156.0,151.0,171.0,158.0,120.0
1,22.0,11.0,17.0,14.0,11.0,24.0,24.0,NaN,155.0,156.0,151.0,171.0,158.0,120.0
2,NaN,35.0,36.0,35.0,38.0,36.0,44.0,NaN,155.0,156.0,151.0,171.0,158.0,120.0
3,NaN,52.0,45.0,53.0,59.0,51.0,54.0,NaN,155.0,156.0,151.0,171.0,158.0,120.0
4,NaN,157.0,177.0,167.0,158.0,164.0,155.0,NaN,155.0,156.0,151.0,171.0,158.0,120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,20.0,32.0,14.0,20.0,36.0,26.0,20.0,25.0,42.0,38.0,47.0,49.0,48.0,29.0
2274,NaN,85.0,86.0,83.0,101.0,91.0,83.0,25.0,42.0,38.0,47.0,49.0,48.0,29.0
2275,20.0,32.0,14.0,20.0,36.0,26.0,20.0,19.0,18.0,15.0,12.0,14.0,16.0,17.0
2276,NaN,85.0,86.0,83.0,101.0,91.0,83.0,19.0,18.0,15.0,12.0,14.0,16.0,17.0


In [47]:
for name in use_systems:
    for prefix,other_prefix in [("opp_","team_"), ("team_","opp_")]:
        try:
            rankings_df[prefix+name+"_higher"]=rankings_df[prefix+name]>rankings_df[other_prefix+name]
            rating_1,rating_2=tourney_rating_mapping(rankings_df[prefix+name]),tourney_rating_mapping(rankings_df[other_prefix+name])
            rankings_df[prefix+name+"_implied_prob"]=predict_prob(rating_1,rating_2)
            rankings_df[prefix+name+"_no_scale_implied_prob"]=0#predict_prob(rating_1,rating_2)
        except:
            continue
reload(utils)
X_,y=utils.transform_to_Xy(fm_encoded_preds, prediction_labels.rename(columns={"game_datetime":"time"}))
X=X_.join(rankings_df)
X[features]

,opp_AP,opp_DOK,opp_POM,opp_SAG,opp_SPR,opp_WOB,team_AP,team_DOK,team_POM,team_SAG,team_SPR,team_WOB,opp_SAG_higher,opp_SAG_implied_prob,opp_SAG_no_scale_implied_prob,team_SAG_higher,team_SAG_implied_prob,team_SAG_no_scale_implied_prob,opp_POM_higher,opp_POM_implied_prob,opp_POM_no_scale_implied_prob,team_POM_higher,team_POM_implied_prob,team_POM_no_scale_implied_prob,opp_DOK_higher,opp_DOK_implied_prob,opp_DOK_no_scale_implied_prob,team_DOK_higher,team_DOK_implied_prob,team_DOK_no_scale_implied_prob,opp_SPR_higher,opp_SPR_implied_prob,opp_SPR_no_scale_implied_prob,team_SPR_higher,team_SPR_implied_prob,team_SPR_no_scale_implied_prob,opp_AP_higher,opp_AP_implied_prob,opp_AP_no_scale_implied_prob,team_AP_higher,team_AP_implied_prob,team_AP_no_scale_implied_prob,opp_WOB_higher,opp_WOB_implied_prob,opp_WOB_no_scale_implied_prob,team_WOB_higher,team_WOB_implied_prob,team_WOB_no_scale_implied_prob
0,17.0,50.0,62.0,53.0,55.0,46.0,NaN,155.0,151.0,171.0,158.0,120.0,False,0.610544,0,True,0.389456,0,False,0.500000,0,True,0.500000,0,False,0.660396,0,True,0.339604,0,False,0.569191,0,True,0.430809,0,False,NaN,0,False,NaN,0,False,0.712159,0,True,0.287841,0
1,22.0,11.0,14.0,11.0,24.0,24.0,NaN,155.0,151.0,171.0,158.0,120.0,False,0.937758,0,True,0.062242,0,False,0.923931,0,True,0.076069,0,False,0.937758,0,True,0.062242,0,False,0.875294,0,True,0.124706,0,False,NaN,0,False,NaN,0,False,0.875294,0,True,0.124706,0
2,NaN,35.0,35.0,38.0,36.0,44.0,NaN,155.0,151.0,171.0,158.0,120.0,False,0.787173,0,True,0.212827,0,False,0.809338,0,True,0.190662,0,False,0.809338,0,True,0.190662,0,False,0.802215,0,True,0.197785,0,False,NaN,0,False,NaN,0,False,0.733742,0,True,0.266258,0
3,NaN,52.0,53.0,59.0,51.0,54.0,NaN,155.0,151.0,171.0,158.0,120.0,False,0.500000,0,True,0.500000,0,False,0.610544,0,True,0.389456,0,False,0.628530,0,True,0.371470,0,False,0.645080,0,True,0.354920,0,False,NaN,0,False,NaN,0,False,0.590871,0,True,0.409129,0
4,NaN,157.0,167.0,158.0,164.0,155.0,NaN,155.0,151.0,171.0,158.0,120.0,False,0.500000,0,True,0.500000,0,True,0.500000,0,False,0.500000,0,True,0.500000,0,False,0.500000,0,True,0.500000,0,False,0.500000,0,False,NaN,0,False,NaN,0,True,0.500000,0,False,0.500000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,20.0,32.0,20.0,36.0,26.0,20.0,25.0,42.0,47.0,49.0,48.0,29.0,False,0.661717,0,True,0.338283,0,False,0.785681,0,True,0.214319,0,False,0.614267,0,True,0.385733,0,False,0.743374,0,True,0.256626,0,False,0.561527,0,True,0.438473,0,False,0.606357,0,True,0.393643,0
2274,NaN,85.0,83.0,101.0,91.0,83.0,25.0,42.0,47.0,49.0,48.0,29.0,True,0.325362,0,False,0.674638,0,True,0.299582,0,False,0.700418,0,True,0.246801,0,False,0.753199,0,True,0.312058,0,False,0.687942,0,False,NaN,0,False,NaN,0,True,0.152343,0,False,0.847657,0
2275,20.0,32.0,20.0,36.0,26.0,20.0,19.0,18.0,12.0,14.0,16.0,17.0,True,0.250339,0,False,0.749661,0,True,0.380418,0,False,0.619582,0,True,0.337347,0,False,0.662653,0,True,0.373449,0,False,0.626551,0,True,0.486733,0,False,0.513267,0,True,0.458992,0,False,0.541008,0
2276,NaN,85.0,83.0,101.0,91.0,83.0,19.0,18.0,12.0,14.0,16.0,17.0,True,0.076069,0,False,0.923931,0,True,0.066848,0,False,0.933152,0,True,0.094819,0,False,0.905181,0,True,0.085368,0,False,0.914632,0,False,NaN,0,False,NaN,0,True,0.090071,0,False,0.909929,0


# ML

In [48]:
from joblib import dump, load
model_name="rankings"
features=load(model_name+"_features.joblib")
clf=load(model_name+'_model.joblib')


In [55]:

X['Pred']=clf.predict_proba(X[features].fillna(0))[:,1]
X['Pred']=X['Pred'].clip(.2,.8)
X['Pred'].describe()

count    2278.000000
mean        0.459123
std         0.203250
min         0.200000
25%         0.283356
50%         0.437231
75%         0.611181
max         0.800000
Name: Pred, dtype: float64

In [56]:
X.rename(columns={"game_index":"ID"})[['ID','Pred']].to_csv("submit-rankings-clipped.csv",index=False)

In [60]:
from bracketeer import build_bracket
b = build_bracket(
        outputPath='2019_rankings_bracket.png',
        teamsPath='../data/DataFiles/Teams.csv',
        seedsPath='../data/DataFiles/NCAATourneySeeds.csv',
        submissionPath='submit-rankings.csv',
        slotsPath='../data/DataFiles/NCAATourneySlots.csv',
        year=2019
)

This is not a great score on Kaggle. Only a small improvement from guessing 50% for each game. The best way to view this matchups_model is as a specialized model that provides information on matchup history when it exists. In the Kaggle submission we have a history based prediction for ~120 out of 335 predictions. Restricting that to teams that have played more than 3 in the last 5 years would reduce the number of informed predictions drastically

# Next Steps
Let me know your thoughts on everything as soon as you can. I think adding seeds would be a useful set of features and be pretty quick since I already have the ID computed in another notebook.    
  
For team_game_logs I need to switch the cutoff_time and may have some questions on that.  
  
Feature groupings could be very interesting but I'm not sure how much I will be able to do if we want to get the article out this weekend. (I think it would be ideal to have the article out by Sunday night in terms of people reading and sharing it but we would need to balance the how many things I can put into it with the time remaining).  
  
I think the article would still be highly relevant into the later rounds of the tournament and there could be a great buzz of interest from the analytics community in the mid-to-late rounds as people discuss the Elite eight final four and eventual champion as they watch their brackets closely. The Kaggle submission is due on Thursday 21st. Good early results could build article interest too.
  
